In [1]:
import pandas as pd
import numpy as np
import math

SOLVER_MILO = "highs"
SOLVER_MINLO = "ipopt"

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["coin", "highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

AMPL Development Version 20240404 (MSVC 19.38.33135.0, 64-bit)
Demo license with maintenance expiring 20260131.
Using license file "c:\Users\thuduong\Anaconda3\envs\optima\Lib\site-packages\ampl_module_base\bin\ampl.lic".



In [2]:
# DATA TEST 1
stocks = {
    "S1": {"width": 1219, "weight": 4395 },
    "S2": {"width": 1219, "weight": 9260},
    "S3": {"width": 1219, "weight": 3475},
    "S4": {"width": 1219, "weight": 8535},
    # "S5": {"width": 236, "weight": 1571},
}

finish = {
    "F1": {"width": 235, "need_cut": 800 },
    "F2": {"width": 147, "need_cut": 1308},
    "F3": {"width": 136, "need_cut": 1290},
    "F4": {"width": 68, "need_cut": 600},
    "F5": {"width": 60, "need_cut": 170},
    "F6": {"width": 85, "need_cut": 132},
    "F7": {"width": 57, "need_cut": 100},
    "F8": {"width": 92, "need_cut": 100}, 
    "F9": {"width": 57, "need_cut": 735}, 
}

In [3]:
## PARAMETER

# over cut bound
BOUND = 0.3
MIN_MARGIN = 8

## Problem with known naive patterns

Given a list of patterns, the optimization problem is to compute how many copies of each pattern should be cut to fit the width of the Mother Coil or 




In [4]:
from codes.create_patterns import *

naive_patterns = make_patterns_by_weight_width(stocks, finish, BOUND, MIN_MARGIN)

# display(naive_patterns)

No feasible pattern was found for Stock S4 and FG F6
No feasible pattern was found for Stock S4 and FG F7
No feasible pattern was found for Stock S4 and FG F8


In [5]:
ap_upper_bound(naive_patterns,finish)

{'F1': 1,
 'F2': 4,
 'F3': 4,
 'F4': 4,
 'F5': 1,
 'F6': 0,
 'F7': 0,
 'F8': 0,
 'F9': 5}

## Pattern Generation: Bilinear Model
Let binary variable $b_{sp}\in\mathbb{Z}_2$ denote the assignment of stock $s$ to pattern $p$, and let $P = 0, 1, \ldots, N_p-1$ index a list of patterns, $wu_{s}$ the weight unit per mm wight of the coil stock, which is the weight divided by the width. For sufficiently large $N_p$, an optimal solution to the stock cutting problem is given by the model

\begin{align}
\min \quad & \sum_{s\in S} b_{sp} l^S_s - \sum_{f\in F}a_{fp}l^F_f \\
\text{s.t.} \quad
& 0.96 (b_{sp} l^S_s) \leq \sum_{f\in F}a_{fp}l^F_f \leq \ b_{sp} l^S_s -\text{margin} &&  \forall s\in S ,\forall p\in P  \\
& wu_{s} \sum_{p\in P}a_{fp} l_{f}  \leq d_f && \forall f\in F\\
& a_{fp} \in \mathbb{Z}_+ && \forall f\in F,  \forall p\in P \\
& b_{sp} \in \{0,1\} && \forall s\in S,  \forall p\in P \\
\end{align}


A solution is attempted using a mixed-integer nonlinear optimization (MINLO) solver.

In [12]:
# problem to find the stock with pattern that fits the weight
#because heavier stock (same width) can also fit the pattern

m = AMPL()
m.option["solver"] = SOLVER_MINLO
m.eval(
    """
        set S;
        set F;
        set P;

        # length stock
        param width_s{S};
        # length finished pieces
        param width_f{F};
        
        # WEIGHT STOCK
        param w{S};

        # weight per unit of stock
        param ws{S};

        # upper bound with over-cut
        param f_upper_demand{F};

        param demand_finish{F};
        param a_upper_bound{F};

        # how many F pieces are returned from pattern p
        var a{f in F, p in P} integer >= 0 <= a_upper_bound[f];
        var b{S, P} binary;

        minimize trim_loss:
          sum{p in P, s in S} b[s,p] * width_s[s] - sum{p in P, f in F} a[f,p] * width_f[f];
          
        subject to feasible_pattern_max_margin{p in P, s in S}:
          sum{f in F} a[f,p] * width_f[f] >= 0.96 * b[s,p] * width_s[s];
        
        subject to feasible_pattern_min_margin{p in P, s in S}:
          sum{f in F} a[f,p] * width_f[f] <= b[s,p] * width_s[s] - 8;

        subject to demand{f in F, p in P, s in S}:
          a[f,p] * width_f[f] * ws[s] <= f_upper_demand[f];
    """
)

In [13]:
m.eval("reset data;")
m.set["S"] = list(stocks.keys())
m.set["F"] = list(finish.keys())
m.set["P"] = list(range(len(naive_patterns))) # list number
s = {p: naive_patterns[p]["stock"] for p in range(len(naive_patterns))} # stock allocated to naive patterns (0, n)
# w = {p: stocks[s[p]]["weight"]/stocks[s[p]]["width"] for p in range(len(patterns))} # weight per unit according to each naive pattern
wstocks = {s: stocks[s]["weight"]/stocks[s]["width"] for s in list(stocks.keys())} # stock weight per unit (unique)
# w.update(wstocks)
m.param["ws"] = wstocks

m.param["width_s"] = {s: stocks[s]["width"] for s in stocks.keys()}
m.param["width_f"] = {f: finish[f]["width"] for f in finish.keys()}
m.param["demand_finish"] = {f: finish[f]["need_cut"] for f in finish.keys()}
m.param["f_upper_demand"] = {f: finish[f]["need_cut"] + BOUND * finish[f]["need_cut"] for f in finish.keys()}

a_upper_bound = { #upper bound of possible cut over all kind of stocks
    f: max([naive_patterns[i]['cuts'][f] for i,_ in enumerate(naive_patterns)])
    for f in finish.keys()
}

m.param["a_upper_bound"] = a_upper_bound
m.get_output("solve;")

trim_loss = m.obj["trim_loss"].value()

In [15]:
# Retrieve the patterns
patterns = []
for p in range(len(finish)):
    a = {f: round(m.var["a"][f, p].value()) for f in finish.keys()}
    patterns.append(
        {
            "stock": [s for s in stocks.keys() if m.var["b"][s, p].value() > 0][0],
            "cuts": a,
        }
    )

In [16]:
patterns

[{'stock': 'S1',
  'cuts': {'F1': 0,
   'F2': 1,
   'F3': 1,
   'F4': 1,
   'F5': 0,
   'F6': 0,
   'F7': 0,
   'F8': 0,
   'F9': 2}},
 {'stock': 'S1',
  'cuts': {'F1': 0,
   'F2': 1,
   'F3': 1,
   'F4': 1,
   'F5': 0,
   'F6': 0,
   'F7': 0,
   'F8': 0,
   'F9': 2}},
 {'stock': 'S1',
  'cuts': {'F1': 0,
   'F2': 1,
   'F3': 1,
   'F4': 1,
   'F5': 0,
   'F6': 0,
   'F7': 0,
   'F8': 0,
   'F9': 2}},
 {'stock': 'S1',
  'cuts': {'F1': 0,
   'F2': 1,
   'F3': 1,
   'F4': 1,
   'F5': 0,
   'F6': 0,
   'F7': 0,
   'F8': 0,
   'F9': 2}},
 {'stock': 'S1',
  'cuts': {'F1': 0,
   'F2': 1,
   'F3': 1,
   'F4': 1,
   'F5': 0,
   'F6': 0,
   'F7': 0,
   'F8': 0,
   'F9': 2}},
 {'stock': 'S1',
  'cuts': {'F1': 0,
   'F2': 1,
   'F3': 1,
   'F4': 1,
   'F5': 0,
   'F6': 0,
   'F7': 0,
   'F8': 0,
   'F9': 2}},
 {'stock': 'S1',
  'cuts': {'F1': 0,
   'F2': 1,
   'F3': 1,
   'F4': 1,
   'F5': 0,
   'F6': 0,
   'F7': 0,
   'F8': 0,
   'F9': 2}},
 {'stock': 'S1',
  'cuts': {'F1': 0,
   'F2': 1,
   'F3

In [11]:
# problem to find the stock with pattern that fits the weight
#because heavier stock (same width) can also fit the pattern

m2 = AMPL()
m2.eval(
        """
        set S;
        set F;
        set P;

        # length stock
        param width_s{S};
        # length finished pieces
        param width_f{F};

        # weight stock
        param w{S}
        # weight per unit of stock
        param wu{S};

        param demand_finish{F};
        param a_upper_bound{F};

        # upper bound with over-cut
        param f_upper_demand{F};

        var a{f in F, p in P} integer >= 0 <= a_upper_bound[f];
        var b{S,P} binary;

        minimize trim_loss_weight:
          sum{p in P, s in S} b[s,p] * w[s] - sum{p in P, s in S} b[s,p] * wu[s] * ( width_s[s] - sum{p in P, f in F} a[f,p] * width_f[f]);

        subject to feasible_stock_to_pattern:
          sum{s in S, p in P} b[s,p] > 0;

        subject to feasible_pattern_max_margin{p in P, s in S}:
          sum{f in F} a[f,p] * width_f[f] >= 0.96 * b[s,p] * width_s[s];
        
        subject to feasible_pattern_min_margin{p in P, s in S}:
          sum{f in F} a[f,p] * width_f[f] <= b[s,p] * width_s[s] - 6;

        subject to demand_in_weight{f in F, p in P, s in S}:
          a[f,p] * width_f[f] * wu[s]<= f_upper_demand[f] ;
    """
)

Error:
	line 14 offset 241
	syntax error
	context:   >>> param  <<< wu{S};

For support/feedback go to https://discuss.ampl.com or e-mail <support@ampl.com>

	line 26 offset 598
	wu is not defined
	context:  sum{p in P, s in S} b[s,p] * w[s] - sum{p in P, s in S} b[s,p] * wu[s]  >>> *  <<< ( width_s[s] - sum{p in P, f in F} a[f,p] * width_f[f]);
Error:
	line 26 offset 600
	syntax error
	context:  sum{p in P, s in S} b[s,p] * w[s] - sum{p in P, s in S} b[s,p] * wu[s] *  >>> ( <<<  width_s[s] - sum{p in P, f in F} a[f,p] * width_f[f]);

For support/feedback go to https://discuss.ampl.com or e-mail <support@ampl.com>

	line 29 offset 743
	Caution: Treating strict inequality constraint as a logical constraint.
	context:  sum{s in S, p in P} b[s,p] >  >>> 0; <<< 
	line 38 offset 1121
	wu is not defined
	context:  a[f,p] * width_f[f] *  >>> wu[s]<=  <<< f_upper_demand[f] ;
Error:
	line 38 offset 1129
	f_upper_demand is already defined
	context:  a[f,p] * width_f[f] * wu[s]<=  >>> f_upper_dem

AMPLException: line 38 offset 1129
f_upper_demand is already defined
context:  a[f,p] * width_f[f] * wu[s]<=  >>> f_upper_demand[ <<< f] ;